<a href="https://colab.research.google.com/github/hirokimituya/book_create-profitable-AI-with-Python/blob/main/technical_analysis/%E9%8A%98%E6%9F%84%E9%81%B8%E5%AE%9A%E9%96%A2%E6%95%B0_csv%E4%BF%9D%E5%AD%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Google Colaboratoryに画像を保存できるように設定
import os
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/株式投資')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Ta-Libの代替としてpandas_taを利用している。Ta-Libの関数名を小文字にしたものがあるイメージ
!pip install pandas_ta

In [3]:
!pip install yahoo_finance_api2

# 銘柄コードから株価情報を取得する関数

In [4]:
from yahoo_finance_api2 import share as yapi2
from yahoo_finance_api2.exceptions import YahooFinanceError
import datetime as dt
from datetime import date,timedelta
import pandas as pd

# 株価データを取得するメソッド
def get_stock_data_yapi2(code, period_type='year', period=10):
    """株価データを取得する

    :param code: 取得する株価データの銘柄コード
    :param period_type: 取得する株価データの期間の単位（例: 'year', 'month', 'week', 'day'）
    :param period: 取得する株価データの期間の数値
    :return: 株価データのデータフレーム（もし株価データが存在しない場合は、None）
    """
    if period_type == 'month':
        period_type_value = yapi2.PERIOD_TYPE_MONTH
    elif period_type == 'week':
        period_type_value = yapi2.PERIOD_TYPE_WEEK
    elif period_type == 'day':
        period_type_value = yapi2.PERIOD_TYPE_DAY
    else:
        period_type_value = yapi2.PERIOD_TYPE_YEAR

    try:
        data = yapi2.Share(f'{code}.T').get_historical(
            period_type_value,
            period,
            yapi2.FREQUENCY_TYPE_DAY,
            1
        )
    except YahooFinanceError as e:
        print(e.message)
        print(f'銘柄コード[{code}]の株価情報を取得できなかったため、スキップしました。')
        return None

    df = pd.DataFrame(data)

    try:
        # タイムスタンプをDateTime型に変更
        df['datetime'] = pd.to_datetime(df['timestamp'], unit='ms')
    except KeyError as e:
        print(e)
        print(f'銘柄コード[{code}]の株価情報を取得できなかったため、スキップしました。')
        return None

    # 日本時間へ変換
    df['Date'] = df['datetime'] + dt.timedelta(hours=9)

    # インデックスをDateカラムに設定
    df.index = pd.to_datetime(df['Date'].dt.date)

    # 不要なカラムを削除
    del df['Date'], df['datetime'], df['timestamp']

    # 各カラムの最初の文字を大文字に変換
    for col in df.columns:
        df[f'{col.capitalize()}'] = df[col]
        del df[col]

    return df

# 東証上場の銘柄一覧を取得する関数（ダウンロードはしない）

In [5]:
from pandas.io.formats.info import DataFrameTableBuilderNonVerbose
import requests
import pandas as pd

def get_ticker_tse(segment='prime'):
    """東証上場の銘柄コード一覧を取得する

    :param segment: 東証上場の市場区分（例: 'prime' or 'standard' or 'grows'）
    :return: 東証上場の銘柄コード一覧
    """
    segments = {
        'prime': 'プライム',
        'standard': 'スタンダード',
        'grows': 'グロース'
    }

    if segment not in segments:
        raise Exception("get_ticker_tse関数のsegment引数の値は'prime', 'standard', 'grows'のいずれかにしてください")

    # サイトからダウンロードしないようにコメントアウト
    # url = "https://www.jpx.co.jp/markets/statistics-equities/misc/tvdivq0000001vg2-att/data_j.xls"
    # r = requests.get(url)
    # with open('./data/data_j.xls', 'wb') as output:
    #    output.write(r.content)

    df = pd.read_excel("./data/data_j.xls")

    # 引数の市場区分のみに絞り込み
    df = df[df['市場・商品区分'].str.startswith(segments[segment])]

    # カラムの削除とカラム名変更
    df = df[['銘柄名', 'コード', '33業種区分']]   # 必要なカラムのみにする
    df = df.rename(columns={'33業種区分': '業種'})
    df['コード'] = df['コード'].astype(int) # コードを整数に変更

    return df

# 株価情報を出来高や株価、パーフェクトオーダーでフィルタリングする関数

In [6]:
import pandas_ta as ta
import math

def is_perfect_order(stock_df, mas, period_type='D', reverse=False):
    """株価情報がパーフェクトオーダーどうかの判定

    :param stock_df: 対象の株価情報のデータフレーム
    :param mas: 移動平均線に使用するためのリスト（例: [5, 25, 75]）
    :param period_type: 期間をどの単位にするか（'W' or 'M'）
    :param reverse: 下降トレンドのパーフェクトオーダーを検出する場合は、True / 上昇トレンドの場合は、False
    :return: 株価情報がパーフェクトオーダーの場合は、True / それ以外の場合は、False
    """

    df = stock_df.copy()

    # 月足、周足にリサンプリング
    if period_type in ('M', 'W'):
        resampled = df.resample(period_type)
        df = resampled.aggregate({'Open': 'first', 'High': 'max', 'Low': 'min', 'Close': 'last', 'Volume': 'sum'})

    close = df['Close']

    # 移動平均線の算出
    for ma in mas:
        df[f'ma{ma}{period_type}'] = ta.sma(close, ma)

    # パーフェクトオーダーかどうかの判定
    conditions = []
    try:
        if not reverse:
            # 上昇トレンド
            for i in range(len(mas) - 1):
                conditions.append(math.ceil(df[f'ma{mas[i]}{period_type}'].values[-1]) >= math.ceil(df[f'ma{mas[i + 1]}{period_type}'].values[-1]))
        else:
            # 下降トレンド
            for i in range(len(mas) - 1):
                conditions.append(math.ceil(df[f'ma{mas[i]}{period_type}'].values[-1]) <= math.ceil(df[f'ma{mas[i + 1]}{period_type}'].values[-1]))
    except:
        # 株価情報が少なくて移動平均線が取得できない場合は、無条件でFalseでリターンする
        return False


    result = all(conditions)

    return result

In [7]:
def filtered_ticker(df, min_volume=None, max_price=None, min_price=None, perfect_order={}):
    """データフレームの銘柄コードから株価情報を取得して、指定された出来高や株価でフィルタリングする

    :param df: 対象のデータフレーム
    :param min_volume: 出来高の最小値を指定
    :param max_price: 株価の最大値を指定
    :param min_price: 株価の最小値を指定
    :param perfect_order: パーフェクトオーダーの判定に使用する辞書（例: {'mas': [5, 20, 60], 'period_type': 'D', 'reverse': False}）（辞書のキー'mas'のみ必須）
    :return: 出来高や株価でフィルタリングしたデータフレーム
    """
    result_list = []

    # データフレームを辞書のリストに変更
    df_to_dicts = df.to_dict(orient='records')

    # 各銘柄についてループ
    for row in df_to_dicts:
        # コードを取得
        code = row['コード']

        # 株価情報を取得
        if not perfect_order:
            stock_data = get_stock_data_yapi2(code, period_type='day', period=2)
        else:
            largest_mas = perfect_order.get('mas')[-1]
            period_type = perfect_order.get('period_type', 'D')
            if period_type == 'D':
                period = math.ceil(largest_mas / 15)
                stock_data = get_stock_data_yapi2(code, period_type='month', period=period)
            elif period_type == 'W':
                period = math.ceil(largest_mas * 7 / 365)
                stock_data = get_stock_data_yapi2(code, period_type='year', period=period)
            elif period_type == 'M':
                period = math.ceil(largest_mas / 10)
                stock_data = get_stock_data_yapi2(code, period_type='year', period=period)
            else:
                raise ValueError('引数perfect_orderは"D"か"W"か"M"のいずれかを指定してください')
        # 株価情報が取得できなかったら、次の銘柄コードに遷移
        if stock_data is None:
            continue

        # 出来高の条件に合わない場合、次の銘柄コードに遷移
        volume = stock_data['Volume'][-1]
        if min_volume is not None and volume < min_volume:
            continue

        # 価格の条件に合わない場合、次の銘柄コードに遷移
        price = stock_data['Close'][-1]
        if max_price is not None and price > max_price:
            continue
        if min_price is not None and price < min_price:
            continue

        # パーフェクトオーダーの条件に合わない場合、次の銘柄コードに遷移
        if perfect_order:
            mas = perfect_order.get('mas')
            period_type = perfect_order.get('period_type', 'D')
            reverse = perfect_order.get('reverse', False)
            if not is_perfect_order(stock_data, mas=mas, period_type=period_type, reverse=reverse):
                continue

        # 銘柄名と業種を設定
        result_list.append({
            '銘柄名': row['銘柄名'],
            'コード': code,
            '業種': row['業種'],
            '出来高': volume,
            '株価': price,
        })

    # 辞書のリストをデータフレームに変換
    result_df = pd.DataFrame.from_dict(result_list)

    return result_df

# CSVに保存する関数

In [8]:
import datetime as dt
import os
import pytz

jst = pytz.timezone('Asia/Tokyo')
now = dt.datetime.now(jst)
now = now.strftime("%Y%m%d_%H%M%S")
# ディレクトリ名作成（YYYYMMDD_HHMMSS_screening）
directory = f'{now}_screening'

def save_df_as_csv(df, filename):
    """データフレームをCSVファイルとして保存する

    :param df: 対象のデータフレーム
    :param filename: 保存するcsvファイル名
    """
    # もしディレクトリが存在しなければ、作成する
    if not os.path.exists(directory):
        os.makedirs(directory)

    # データフレームをCSVに保存する
    df.to_csv(f'{directory}/{filename}.csv', index=False)

# 東証プライムをフィルタリングしてCSVとして保存

In [9]:
# 東証プライム市場の銘柄コードを取得
tse_prime = get_ticker_tse('prime')
tse_prime

,銘柄名,コード,業種
0,極洋,1301,水産・農林業
16,ニッスイ,1332,水産・農林業
17,マルハニチロ,1333,水産・農林業
33,雪国まいたけ,1375,水産・農林業
35,サカタのタネ,1377,水産・農林業
...,...,...,...
4272,サンドラッグ,9989,小売業
4273,サックスバー ホールディングス,9990,小売業
4274,ジェコス,9991,卸売業
4277,グローセル,9995,卸売業


In [10]:
# 東証プライムの銘柄コードを以下の条件に合致するもののみにフィルタリング
# 出来高　... 50万株以上
# 株価 ... １００円以上、5000円以下
import time
start_time = time.time()
filtered_tse_prime = filtered_ticker(tse_prime, min_volume=500000, min_price=100, max_price=5000)
print(f'経過時間: {time.time() - start_time}秒')
filtered_tse_prime

Not Found: No data found, symbol may be delisted
銘柄コード[25935]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[50749]の株価情報を取得できなかったため、スキップしました。
経過時間: 348.3619318008423秒


,銘柄名,コード,業種,出来高,株価
0,ニッスイ,1332,水産・農林業,3325500,762.500000
1,ＩＮＰＥＸ,1605,鉱業,5029500,2007.500000
2,オリエンタル白石,1786,建設業,806900,322.000000
3,大成建設,1801,建設業,539200,4826.000000
4,大林組,1802,建設業,1053300,1285.500000
...,...,...,...,...,...
296,エイチ・アイ・エス,9603,サービス業,627300,1997.000000
297,ＮＴＴデータグループ,9613,情報・通信業,2576900,1881.500000
298,ヤマダホールディングス,9831,小売業,3863900,465.200012
299,吉野家ホールディングス,9861,小売業,712200,2772.500000


In [11]:
# 株価情報が上昇トレンドのパーフェクトオーダー（週足）かどうかの判断
filtered_perfect_order_week_tse_prime = filtered_ticker(filtered_tse_prime, perfect_order={'mas': [5, 20], 'period_type': 'W'})
filtered_perfect_order_week_tse_prime

,銘柄名,コード,業種,出来高,株価
0,ニッスイ,1332,水産・農林業,3325500,762.500000
1,ＩＮＰＥＸ,1605,鉱業,5029500,2007.500000
2,オリエンタル白石,1786,建設業,806900,322.000000
3,大成建設,1801,建設業,539200,4826.000000
4,大林組,1802,建設業,1053300,1285.500000
...,...,...,...,...,...
207,東京瓦斯,9531,電気・ガス業,852500,3310.000000
208,大阪瓦斯,9532,電気・ガス業,1193100,2277.500000
209,エイチ・アイ・エス,9603,サービス業,627300,1997.000000
210,ヤマダホールディングス,9831,小売業,3863900,465.200012


In [12]:
# 株価情報が上昇トレンドのパーフェクトオーダー（日足）かどうかの判断
filtered_perfect_order_week_day_tse_prime = filtered_ticker(filtered_perfect_order_week_tse_prime,  perfect_order={'mas': [5, 20, 60]})
filtered_perfect_order_week_day_tse_prime

,銘柄名,コード,業種,出来高,株価
0,ニッスイ,1332,水産・農林業,3325500,762.500000
1,ＩＮＰＥＸ,1605,鉱業,5029500,2007.500000
2,オリエンタル白石,1786,建設業,806900,322.000000
3,鹿島建設,1812,建設業,1622200,2360.000000
4,三井住友建設,1821,建設業,795700,397.000000
...,...,...,...,...,...
107,東北電力,9506,電気・ガス業,1794300,994.500000
108,電源開発,9513,電気・ガス業,609300,2258.000000
109,東京瓦斯,9531,電気・ガス業,852500,3310.000000
110,大阪瓦斯,9532,電気・ガス業,1193100,2277.500000


In [13]:
# CSVファイル保存
save_df_as_csv(filtered_perfect_order_week_day_tse_prime, '1_東証P_UPO')

In [14]:
# 株価情報が下降トレンドのパーフェクトオーダー（週足）かどうかの判断
filtered_reverse_perfect_order_week_tse_prime = filtered_ticker(filtered_tse_prime, perfect_order={'mas': [5, 20], 'period_type': 'W', 'reverse': True})
filtered_reverse_perfect_order_week_tse_prime

,銘柄名,コード,業種,出来高,株価
0,ＬＩＦＵＬＬ,2120,サービス業,891300,229.000000
1,日本Ｍ＆Ａセンターホールディングス,2127,サービス業,5682500,789.700012
2,パーソルホールディングス,2181,サービス業,577700,2516.500000
3,システナ,2317,情報・通信業,993300,264.000000
4,日本駐車場開発,2353,不動産業,1681500,219.000000
...,...,...,...,...,...
84,ＫＤＤＩ,9433,情報・通信業,3306100,4270.000000
85,イーレックス,9517,電気・ガス業,552400,939.000000
86,レノバ,9519,電気・ガス業,1006100,1402.000000
87,ＮＴＴデータグループ,9613,情報・通信業,2576900,1881.500000


In [15]:
# 株価情報が下降トレンドのパーフェクトオーダー（日足）かどうかの判断
filtered_reverse_perfect_order_week_day_tse_prime = filtered_ticker(filtered_reverse_perfect_order_week_tse_prime, perfect_order={'mas': [5, 20, 60], 'reverse': True})
filtered_reverse_perfect_order_week_day_tse_prime

,銘柄名,コード,業種,出来高,株価
0,ＬＩＦＵＬＬ,2120,サービス業,891300,229.0
1,パーソルホールディングス,2181,サービス業,577700,2516.5
2,カカクコム,2371,サービス業,797100,1707.5
3,ベネフィット・ワン,2412,サービス業,567400,1211.0
4,エムスリー,2413,サービス業,1893000,2806.0
...,...,...,...,...,...
64,日本通信,9424,情報・通信業,1977400,229.0
65,イーレックス,9517,電気・ガス業,552400,939.0
66,レノバ,9519,電気・ガス業,1006100,1402.0
67,ＮＴＴデータグループ,9613,情報・通信業,2576900,1881.5


In [16]:
# CSVファイル保存
save_df_as_csv(filtered_reverse_perfect_order_week_day_tse_prime, '4_東証P_DPO')

# 東証スタンダードをフィルタリングしてCSVとして保存

In [17]:
# 東証スタンダード市場の銘柄コードを取得
tse_standard = get_ticker_tse('standard')
tse_standard

,銘柄名,コード,業種
34,カネコ種苗,1376,水産・農林業
37,秋川牧園,1380,水産・農林業
38,アクシーズ,1381,水産・農林業
39,ホーブ,1382,水産・農林業
40,ベルグアース,1383,水産・農林業
...,...,...,...
4267,タキヒヨー,9982,卸売業
4270,蔵王産業,9986,卸売業
4275,ヤマザワ,9993,小売業
4276,やまや,9994,小売業


In [18]:
# 東証スタンダードの銘柄コードを以下の条件に合致するもののみにフィルタリング
# 出来高　... 20万株以上
# 株価 ... 100円以上、5000円以下
filtered_tse_standard = filtered_ticker(tse_standard, min_volume=200000, min_price=100, max_price=5000)
filtered_tse_standard

'timestamp'
銘柄コード[92649]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[9967]の株価情報を取得できなかったため、スキップしました。


,銘柄名,コード,業種,出来高,株価
0,ウエストホールディングス,1407,建設業,273100,2604.000000
1,住石ホールディングス,1514,鉱業,253000,360.000000
2,塩水港精糖,2112,食料品,345800,259.000000
3,フジ日本精糖,2114,食料品,202600,1089.000000
4,クルーズ,2138,情報・通信業,340400,1001.000000
...,...,...,...,...,...
65,アコム,8572,その他金融業,896400,340.600006
66,トレイダーズホールディングス,8704,証券、商品先物取引業,211900,738.000000
67,小林洋行,8742,証券、商品先物取引業,402000,227.000000
68,ノバレーゼ,9160,サービス業,204600,393.000000


In [19]:
# 株価情報が上昇トレンドのパーフェクトオーダー（週足）かどうかの判断
filtered_perfect_order_week_tse_standard = filtered_ticker(filtered_tse_standard, perfect_order={'mas': [5, 20], 'period_type': 'W'})
filtered_perfect_order_week_tse_standard

,銘柄名,コード,業種,出来高,株価
0,塩水港精糖,2112,食料品,345800,259.000000
1,フジ日本精糖,2114,食料品,202600,1089.000000
2,クルーズ,2138,情報・通信業,340400,1001.000000
3,ＹＥ ＤＩＧＩＴＡＬ,2354,情報・通信業,282000,650.000000
4,鉄人化計画,2404,サービス業,200800,407.000000
5,Ｓｈｉｎｗａ Ｗｉｓｅ Ｈｏｌｄｉｎｇｓ,2437,サービス業,203900,618.000000
6,出前館,2484,情報・通信業,603400,408.000000
7,シー・ヴイ・エス・ベイエリア,2687,小売業,419600,1030.000000
8,アップルインターナショナル,2788,卸売業,486300,434.000000
9,ポラリス・ホールディングス,3010,サービス業,842000,171.000000


In [20]:
# 株価情報が上昇トレンドのパーフェクトオーダー（日足）かどうかの判断
filtered_perfect_order_week_day_tse_standard = filtered_ticker(filtered_perfect_order_week_tse_standard,  perfect_order={'mas': [5, 20, 60]})
filtered_perfect_order_week_day_tse_standard

,銘柄名,コード,業種,出来高,株価
0,塩水港精糖,2112,食料品,345800,259.0
1,フジ日本精糖,2114,食料品,202600,1089.0
2,クルーズ,2138,情報・通信業,340400,1001.0
3,ＹＥ ＤＩＧＩＴＡＬ,2354,情報・通信業,282000,650.0
4,鉄人化計画,2404,サービス業,200800,407.0
5,シー・ヴイ・エス・ベイエリア,2687,小売業,419600,1030.0
6,アップルインターナショナル,2788,卸売業,486300,434.0
7,ポラリス・ホールディングス,3010,サービス業,842000,171.0
8,ダイドーリミテッド,3205,繊維製品,375000,301.0
9,ＴＨＥグローバル社,3271,不動産業,213400,422.0


In [21]:
# CSVファイル保存
save_df_as_csv(filtered_perfect_order_week_day_tse_standard, '2_東証S_UPO')

In [22]:
# 株価情報が下降トレンドのパーフェクトオーダー（週足）かどうかの判断
filtered_reverse_perfect_order_week_tse_standard = filtered_ticker(filtered_tse_standard, perfect_order={'mas': [5, 20], 'period_type': 'W', 'reverse': True})
filtered_reverse_perfect_order_week_tse_standard

,銘柄名,コード,業種,出来高,株価
0,ウエストホールディングス,1407,建設業,273100,2604.0
1,住石ホールディングス,1514,鉱業,253000,360.0
2,クシム,2345,情報・通信業,351100,313.0
3,イメージ ワン,2667,卸売業,665100,622.0
4,セリア,2782,小売業,204400,2290.0
5,ＪＦＬＡホールディングス,3069,小売業,361200,194.0
6,白鳩,3192,小売業,322400,340.0
7,ユナイテッド・スーパーマーケット・ホールディングス,3222,小売業,241300,1133.0
8,セルシス,3663,情報・通信業,343000,745.0
9,ｅｎｉｓｈ,3667,情報・通信業,378000,256.0


In [23]:
# 株価情報が下降トレンドのパーフェクトオーダー（日足）かどうかの判断
filtered_reverse_perfect_order_week_day_tse_standard = filtered_ticker(filtered_reverse_perfect_order_week_tse_standard,  perfect_order={'mas': [5, 20, 60], 'reverse': True})
filtered_reverse_perfect_order_week_day_tse_standard

,銘柄名,コード,業種,出来高,株価
0,ウエストホールディングス,1407,建設業,273100,2604.0
1,クシム,2345,情報・通信業,351100,313.0
2,イメージ ワン,2667,卸売業,665100,622.0
3,ＪＦＬＡホールディングス,3069,小売業,361200,194.0
4,ｅｎｉｓｈ,3667,情報・通信業,378000,256.0
5,ブロードバンドタワー,3776,情報・通信業,276400,143.0
6,リミックスポイント,3825,情報・通信業,309100,223.0
7,ハーモニック・ドライブ・システムズ,6324,機械,205100,3630.0
8,フェローテックホールディングス,6890,電気機器,585200,3030.0
9,ヘリオス テクノ ホールディング,6927,電気機器,492800,440.0


In [24]:
# CSVファイル保存
save_df_as_csv(filtered_reverse_perfect_order_week_day_tse_standard, '5_東証S_DPO')

# 東証グロースをフィルタリングしてCSVとして保存

In [25]:
# 東証スタンダード市場の銘柄コードを取得
tse_grows = get_ticker_tse('grows')
tse_grows

,銘柄名,コード,業種
45,ルーデン・ホールディングス,1400,建設業
46,エムビーエス,1401,建設業
55,Ｌｉｂ Ｗｏｒｋ,1431,建設業
60,フィット,1436,建設業
65,ニッソウ,1444,建設業
...,...,...,...
4091,プログリット,9560,サービス業
4092,グラッドキューブ,9561,サービス業
4093,ビジネスコーチ,9562,サービス業
4094,Ａｔｌａｓ Ｔｅｃｈｎｏｌｏｇｉｅｓ,9563,サービス業


In [26]:
# 東証グロースの銘柄コードを以下の条件に合致するもののみにフィルタリング
# 出来高　... 20万株以上
# 株価 ... １００円以上、5000円以下
filtered_tse_grows = filtered_ticker(tse_grows, min_volume=200000, min_price=100, max_price=5000)
filtered_tse_grows

'timestamp'
銘柄コード[3782]の株価情報を取得できなかったため、スキップしました。


,銘柄名,コード,業種,出来高,株価
0,ケアネット,2150,サービス業,281200,869.0
1,ＦＲＯＮＴＥＯ,2158,サービス業,312800,680.0
2,ジーエヌアイグループ,2160,医薬品,9971900,2095.0
3,ウェッジホールディングス,2388,その他金融業,523200,147.0
4,アマナ,2402,サービス業,538400,553.0
...,...,...,...,...,...
83,スマサポ,9342,サービス業,306600,1890.0
84,リニューアブル・ジャパン,9522,電気・ガス業,2073200,1147.0
85,Ｍ＆Ａ総研ホールディングス,9552,サービス業,678200,3500.0
86,マイクロアド,9553,サービス業,525900,3035.0


In [27]:
# 株価情報が上昇トレンドのパーフェクトオーダー（週足）かどうかの判断
filtered_perfect_order_week_tse_grows = filtered_ticker(filtered_tse_grows, perfect_order={'mas': [5, 20], 'period_type': 'W'})
filtered_perfect_order_week_tse_grows

,銘柄名,コード,業種,出来高,株価
0,ＦＲＯＮＴＥＯ,2158,サービス業,312800,680.0
1,ジーエヌアイグループ,2160,医薬品,9971900,2095.0
2,ウェッジホールディングス,2388,その他金融業,523200,147.0
3,ベースフード,2936,食料品,365400,579.0
4,ソーシャルワイヤー,3929,情報・通信業,1013100,327.0
5,シェアリングテクノロジー,3989,情報・通信業,843200,718.0
6,アララ,4015,情報・通信業,1249700,784.0
7,ヤプリ,4168,情報・通信業,396500,1442.0
8,ＷＡＣＵＬ,4173,情報・通信業,496400,989.0
9,スパイダープラス,4192,情報・通信業,223300,685.0


In [28]:
# 株価情報が上昇トレンドのパーフェクトオーダー（日足）かどうかの判断
filtered_perfect_order_week_day_tse_grows = filtered_ticker(filtered_perfect_order_week_tse_grows,  perfect_order={'mas': [5, 20, 60]})
filtered_perfect_order_week_day_tse_grows

,銘柄名,コード,業種,出来高,株価
0,ジーエヌアイグループ,2160,医薬品,9971900,2095.0
1,ベースフード,2936,食料品,365400,579.0
2,アララ,4015,情報・通信業,1249700,784.0
3,ヤプリ,4168,情報・通信業,396500,1442.0
4,ＷＡＣＵＬ,4173,情報・通信業,496400,989.0
5,ＨＥＮＮＧＥ,4475,情報・通信業,693900,1003.0
6,ＢＡＳＥ,4477,情報・通信業,2716600,323.0
7,免疫生物研究所,4570,医薬品,5652400,800.0
8,メドレックス,4586,医薬品,8340900,264.0
9,プレミアアンチエイジング,4934,化学,322100,1289.0


In [29]:
# CSVファイル保存
save_df_as_csv(filtered_perfect_order_week_day_tse_grows, '3_東証G_UPO')

In [30]:
# 株価情報が下降トレンドのパーフェクトオーダー（週足）かどうかの判断
filtered_reverse_perfect_order_week_tse_grows = filtered_ticker(filtered_tse_grows, perfect_order={'mas': [5, 20], 'period_type': 'W', 'reverse': True})
filtered_reverse_perfect_order_week_tse_grows

,銘柄名,コード,業種,出来高,株価
0,ケアネット,2150,サービス業,281200,869.0
1,アマナ,2402,サービス業,538400,553.0
2,海帆,3133,小売業,2040900,397.0
3,アクセルマーク,3624,情報・通信業,4439700,255.0
4,Ａｉｍｉｎｇ,3911,情報・通信業,12829500,404.0
5,プレイド,4165,情報・通信業,537900,636.0
6,ＥＮＥＣＨＡＮＧＥ,4169,情報・通信業,557600,1141.0
7,エクサウィザーズ,4259,情報・通信業,630800,429.0
8,セーフィー,4375,情報・通信業,414100,870.0
9,ＪＤＳＣ,4418,情報・通信業,716400,952.0


In [31]:
# 株価情報が下降トレンドのパーフェクトオーダー（日足）かどうかの判断
filtered_reverse_perfect_order_week_day_tse_grows = filtered_ticker(filtered_reverse_perfect_order_week_tse_grows,  perfect_order={'mas': [5, 20, 60], 'reverse': True})
filtered_reverse_perfect_order_week_day_tse_grows

,銘柄名,コード,業種,出来高,株価
0,ケアネット,2150,サービス業,281200,869.0
1,海帆,3133,小売業,2040900,397.0
2,アクセルマーク,3624,情報・通信業,4439700,255.0
3,プレイド,4165,情報・通信業,537900,636.0
4,ＥＮＥＣＨＡＮＧＥ,4169,情報・通信業,557600,1141.0
5,エクサウィザーズ,4259,情報・通信業,630800,429.0
6,Ｃｈａｔｗｏｒｋ,4448,情報・通信業,414600,834.0
7,フリー,4478,情報・通信業,882500,2728.0
8,キャンバス,4575,医薬品,313700,982.0
9,キッズウェル・バイオ,4584,医薬品,234700,184.0


In [32]:
# CSVファイル保存
save_df_as_csv(filtered_reverse_perfect_order_week_day_tse_grows, '6_東証G_DPO')

# 東証スタンダード・プライム選抜をCSVとして保存

In [33]:
import pandas as pd

def remove_duplicate_rows(dataframe, *other_dataframes):
    """重複しているティッカーコードの行を削除する

    :param dataframe: 対象のデータフレーム
    :param other_dataframes: 重複しているかどうかの判断のためのデータフレーム
    :return 引数dataframeで重複行を削除したもの
    """
    # データフレームをコピーして新しいデータフレームを作成
    result_df = dataframe.copy()

    # 重複確認のためのセットを作成
    unique_values = set()
    for df in other_dataframes:
        unique_values.update(df['コード'])

    # 重複しているティッカーコードを削除
    result_df = result_df[~result_df['コード'].isin(unique_values)]

    return result_df

In [34]:
# 東証スタンダードの銘柄コードを以下の条件に合致するもののみにフィルタリング
# 出来高　... 50万株以上
# 株価 ... １００円以上、5000円以下
filtered_tse_standard_2 = filtered_ticker(tse_standard, min_volume=500000, min_price=100, max_price=5000)
filtered_tse_standard_2

'timestamp'
銘柄コード[92649]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[9967]の株価情報を取得できなかったため、スキップしました。


,銘柄名,コード,業種,出来高,株価
0,出前館,2484,情報・通信業,603400,408.000000
1,イメージ ワン,2667,卸売業,665100,622.000000
2,ポラリス・ホールディングス,3010,サービス業,842000,171.000000
3,ビューティ花壇,3041,卸売業,1056700,1094.000000
4,ジェイ・エスコムホールディングス,3779,情報・通信業,6214600,173.000000
5,ＰＫＳＨＡ Ｔｅｃｈｎｏｌｏｇｙ,3993,情報・通信業,501600,2585.000000
6,アテクト,4241,化学,1002300,764.000000
7,セプテーニ・ホールディングス,4293,サービス業,514000,414.000000
8,環境管理センター,4657,サービス業,1124100,614.000000
9,アトラグループ,6029,サービス業,1520200,272.000000


In [35]:
# ティッカーコードが重複している行を削除
filtered_tse_standard_2_remove_duplicate = remove_duplicate_rows(filtered_tse_standard_2, filtered_perfect_order_week_day_tse_standard, filtered_reverse_perfect_order_week_day_tse_standard)
filtered_tse_standard_2_remove_duplicate

,銘柄名,コード,業種,出来高,株価
0,出前館,2484,情報・通信業,603400,408.000000
3,ビューティ花壇,3041,卸売業,1056700,1094.000000
5,ＰＫＳＨＡ Ｔｅｃｈｎｏｌｏｇｙ,3993,情報・通信業,501600,2585.000000
10,シライ電子工業,6658,電気機器,512800,568.000000
13,ＧＭＢ,7214,輸送用機器,1113900,1900.000000
15,テイツー,7610,小売業,897300,188.000000
17,クボテック,7709,精密機器,1955300,307.000000
19,アコム,8572,その他金融業,896400,340.600006
20,日本テレホン,9425,情報・通信業,591900,355.000000


In [36]:
# CSVファイル保存
save_df_as_csv(filtered_tse_standard_2_remove_duplicate, '7_東証S_選抜')

In [37]:
# 東証グロースの銘柄コードを以下の条件に合致するもののみにフィルタリング
# 出来高　... 50万株以上
# 株価 ... １００円以上、5000円以下
filtered_tse_grows_2 = filtered_ticker(tse_grows, min_volume=500000, min_price=100, max_price=5000)
filtered_tse_grows_2

'timestamp'
銘柄コード[3782]の株価情報を取得できなかったため、スキップしました。


,銘柄名,コード,業種,出来高,株価
0,ジーエヌアイグループ,2160,医薬品,9971900,2095.0
1,ウェッジホールディングス,2388,その他金融業,523200,147.0
2,アマナ,2402,サービス業,538400,553.0
3,海帆,3133,小売業,2040900,397.0
4,アクセルマーク,3624,情報・通信業,4439700,255.0
5,Ａｉｍｉｎｇ,3911,情報・通信業,12829500,404.0
6,ソーシャルワイヤー,3929,情報・通信業,1013100,327.0
7,シェアリングテクノロジー,3989,情報・通信業,843200,718.0
8,アララ,4015,情報・通信業,1249700,784.0
9,プレイド,4165,情報・通信業,537900,636.0


In [38]:
# ティッカーコードが重複している行を削除
filtered_tse_grows_2_remove_duplicate = remove_duplicate_rows(filtered_tse_grows_2, filtered_perfect_order_week_day_tse_grows, filtered_reverse_perfect_order_week_day_tse_grows)
filtered_tse_grows_2_remove_duplicate

,銘柄名,コード,業種,出来高,株価
1,ウェッジホールディングス,2388,その他金融業,523200,147.0
2,アマナ,2402,サービス業,538400,553.0
5,Ａｉｍｉｎｇ,3911,情報・通信業,12829500,404.0
6,ソーシャルワイヤー,3929,情報・通信業,1013100,327.0
7,シェアリングテクノロジー,3989,情報・通信業,843200,718.0
12,ＪＤＳＣ,4418,情報・通信業,716400,952.0
16,アンジェス,4563,医薬品,1916700,104.0
22,カバー,5253,情報・通信業,2109600,2171.0
25,Ｇｌｏｂｅｅ,5575,情報・通信業,638700,2138.0
26,ＡＲアドバンストテクノロジ,5578,情報・通信業,530300,2810.0


In [39]:
# CSVファイル保存
save_df_as_csv(filtered_tse_grows_2_remove_duplicate, '8_東証G_選抜')